In [ ]:
import os
CASSANDRA_IP=os.getenv('CASSANDRA1')
print(CASSANDRA_IP)

if CASSANDRA_IP is None:
    CASSANDRA_IP = '172.18.0.2'

from cassandra.cluster import Cluster
cluster = Cluster([CASSANDRA_IP])
session = cluster.connect()
session.execute('DROP KEYSPACE IF EXISTS classroom')
session.execute("CREATE KEYSPACE classroom WITH REPLICATION={'class':'SimpleStrategy', 'replication_factor':'1'}")
session = cluster.connect('classroom')
session.execute("create table student(id int PRIMARY KEY, firstname text, lastname text, emails set<text>)")
session.execute("insert into student (id, firstname, lastname, emails) values (1, 'Joe', 'Smith', {'joes@xyz.com', 'joe.smith@abc.net'})")
session.execute("update student set firstname = 'Joseph' where id = 1")
session.execute("insert into student (id, firstname, lastname, emails) values (2, 'Mike', 'Jones', {'mikej@xyz.com', 'mike.jones@def.net', 'mike1234@gmail.com'})")
rows = session.execute('SELECT id, firstname, lastname, emails from student')
print(list(rows))


### In order for Spark to talk to Cassandra it needs to know the IP address to initialize the spark context with and it also needs the spark-cassandra-connector.

### Mongo will be similar and we need to initial the spark context with pointers to the mongo uri and also include the mongo-spark-connector

### Additionally, whoever configures the cluster may need to make sure additional jars are installed in $SPARK_HOME/jars

Don't run the following, it is just included here to have a look at.

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.1,com.datastax.spark:spark-cassandra-connector_2.11:2.4.0 pyspark-shell'

def initspark(appname = "Test", servername = "local", cassandra="127.0.0.1", mongo="mongodb://127.0.0.1/classroom"):
    print ('initializing pyspark')
    conf = SparkConf().set("spark.cassandra.connection.host", cassandra).setAppName(appname).setMaster(servername)
    sc = SparkContext(conf=conf)
    spark = SparkSession.builder.appName(appname) \
    .config("spark.mongodb.input.uri", mongo) \
    .config("spark.mongodb.output.uri", mongo) \
    .enableHiveSupport().getOrCreate()
    sc.setLogLevel("WARN")
    print ('pyspark initialized')
    return sc, spark, conf


### Let's initialize as usual but pass in the IP address of the Cassandra cluster this time

In [ ]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark(cassandra=CASSANDRA_IP)


### Using the spark-cassandra-connector we can read from a cassandra table in a similar way to how we read from a MySQL table

In [ ]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)
print(people.collect())


### The results of a DataFrame can also be written to a Cassandra table

In [ ]:
# Append the results of a DataFrame into a Cassandra table
x = sc.parallelize([(3, 'Mary', 'Johnson', ['Mary1@gmail.com', 'Mary2@yahoo.com'])])
x1 = spark.createDataFrame(x, schema = ['id', 'firstname', 'lastname', 'emails'])
x1.write.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").mode("append").save()
print('Done')

In [ ]:
people = spark.read.format("org.apache.spark.sql.cassandra").options(table="student", keyspace="classroom").load()
display(people)


### Once you have a DataFrame pointing to a cassandra table, you can use normal SparkSQL on it by making it a temporary view. Here we will flatten out the list of emails using LATERAL VIEW EXPLODE

In [ ]:
people.createOrReplaceTempView('people')
people2 = spark.sql('select id, firstname, lastname, email from people LATERAL VIEW EXPLODE(emails) EXPLODED_TABLE AS email')
display(people2)


### Alternatively you could also use dot syntax and make method calls

In [ ]:
people3 = people2.where("email like '%.com'").orderBy("id")
display(people3)

### Python directly talking to Mongo without Spark

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
classroom = client["classroom"]
if 'classroom' in (x['name'] for x in client.list_databases()):
    client.drop_database('classroom')

people = classroom['people']
name = {"firstname" : "Adam", "personid":4}
x = people.insert_one(name)

names = [{"firstname" : "Betty", "personid":5}
         ,{"firstname" : "Charlie", "personid":6}]
x = people.insert_many(names)

x = people.find()
print ('*' * 80)
print ('from mongo directly')
print (list(x))
print ('*' * 80)



In [ ]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


### Like Cassandra before, we can take a DataFrame and write it to a Mongo destination

In [ ]:
x = sc.parallelize([(7, 'David')])
x1 = spark.createDataFrame(x, schema = ['personid', 'firstname'])
x1.write.format("mongo").options(collection="people", database="classroom").mode("append").save()
print('Done')


In [ ]:
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/classroom.people").load()
df.show()


### Like any DataFrame, we can make it into a temporary view and use SparkSQL on it

In [ ]:
df.createOrReplaceTempView('people')
spark.sql('select * from people').show()
